## Download html files

request,bsでは動的コンテンツをとってこれないのでseleniumを使う

In [2]:
import requests

In [3]:
url="https://toho-vote.info/result"

In [4]:
r=requests.get(url)

In [5]:
r

<Response [200]>

In [ ]:
r.content

In [ ]:
r.write()

## htmlからcsvファイル作成(scraping)

In [4]:
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

In [16]:
ddir="data"
filename_tr="/18/第18回東方Project人気投票_人妖"
filename=ddir+filename_tr+".html"

In [5]:
with open(ddir+filename,"r") as fp:
    htmltext=fp.read()
soup = BeautifulSoup(htmltext, "html.parser")

ブラウザのデバッガ
document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
の結果から

In [4]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.div

<div class="_title_en_1h8e9_51">Characters</div>

In [5]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header

<header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header>

In [6]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]

In [7]:
table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div


In [8]:
table_header=table.find_all("th")
table_contents=table.find_all("tr")

In [9]:
table_header

[<th>順位</th>,
 <th class="_rank_prev_1orw9_133">前回</th>,
 <th class="_rank_prev2_1orw9_136">前々</th>,
 <th class="_name_1orw9_101">名前</th>,
 <th>ポイント</th>,
 <th>一推し</th>,
 <th>コメント</th>,
 <th>支援作品</th>]

In [21]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]    
def get_tableheader(table):
    return [[t.string for t in tr.find_all("th")] for tr in table]    

In [12]:
outdata=get_tabledata(table_contents)

In [14]:
outdata[0]=outheader
outdata[:2]

[[], ['1', '5', '5', 'フランドール・スカーレット', '16,723', '3,730', '3,633', '19']]

### output

In [30]:
import csv
with open(ddir+filename_tr+".csv", 'w') as fpw:
    writer = csv.writer(fpw, lineterminator='\n')
    writer.writerows(outdata)
    

In [31]:
#filename_tr="/17/第17回東方Project人気投票_人妖"
filename_tr="/17/人妖"

def converthtml2csv(filename_tr):
    filename=ddir+filename_tr+".html"
    with open(filename,"r") as fp:
        htmltext=fp.read()

    soup = BeautifulSoup(htmltext, "html.parser")
    table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div

    table_header=table.find_all("th")
    table_contents=table.find_all("tr")

    outdata  =get_tabledata(table_contents)
    outheader=get_tableheader(table_header)
    outdata[0]=outheader

    with open(ddir+filename_tr+".csv", 'w') as fpw:
        writer = csv.writer(fpw, lineterminator='\n')
        writer.writerows(outdata)

In [27]:
table_header

[<th>順位</th>,
 <th class="_rank_prev_1orw9_133">前回</th>,
 <th class="_rank_prev2_1orw9_136">前々</th>,
 <th class="_name_1orw9_101">名前</th>,
 <th>ポイント</th>,
 <th>一推し</th>,
 <th>コメント</th>,
 <th>支援作品</th>]